In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df1 = pd.read_excel("fake_data.xlsx",sheet_name='data_1',dtype='object')
df2 = pd.read_excel("fake_data.xlsx",sheet_name='data_2',dtype='object')

In [3]:
merge_df = df1.merge(df2,on='customer_id')

In [4]:
[col for col in merge_df.columns if '_x' in col or '_y'in col]

['age_x',
 'dob_x',
 'bank_code_x',
 'balance_amount_x',
 'age_y',
 'dob_y',
 'bank_code_y',
 'balance_amount_y']

In [5]:
for col in merge_df.columns:
    if col[-2:] == ('_x'):
        a = col
        col_nam = '_'.join(col.split('_')[:-1])
        if col_nam + '_y' in merge_df.columns:
            b = col_nam + '_y'
            t_f = (merge_df[a] == merge_df[b]).sum() == len(merge_df)
            print(f"{col_nam} > > with _x, _y : {t_f}")
            if t_f == True:
                merge_df[col_nam] = merge_df[a]
                merge_df.drop([a, b], axis=1, inplace=True)


age > > with _x, _y : False
dob > > with _x, _y : False
bank_code > > with _x, _y : True
balance_amount > > with _x, _y : False


In [6]:
# set([val[0] for val in merge_df.cust_name])
one = ['A', 'B', 'C', 'D']
two = ['H', 'J', 'L', 'M']
three = ['O', 'R', 'S', 'T', 'W']

In [20]:
for index, row in merge_df.iterrows():
    if row['cust_name'][0] in one:
        merge_df.at[index, 'one'] = 1
    elif row['cust_name'][0] in two:
        merge_df.at[index, 'two'] = 1
    elif row['cust_name'][0] in three:
        merge_df.at[index, 'three'] = 1

In [22]:
for index, row in merge_df.iterrows():
    if row['cust_name'][0] in one:
        merge_df.at[index, 'type'] = 'one'
    elif row['cust_name'][0] in two:
        merge_df.at[index, 'type'] = 'two'
    elif row['cust_name'][0] in three:
        merge_df.at[index, 'type'] = 'three'

In [11]:
type_in_table = [typ for typ in merge_df.type.unique() if typ in merge_df.columns]

In [12]:
merge_df[['three', 'two', 'one']] = merge_df[['three', 'two', 'one']].fillna(0)

In [13]:
type_dict = merge_df.type.value_counts().to_dict()

In [15]:
for typ in type_in_table:
    if typ in merge_df.columns:
        print(f"{typ} - {merge_df[typ].astype(int).sum()}")
    else:
        print(f"{typ} - this column doesn't exist")

three - 7
two - 10
one - 8


In [16]:
type_dict.items()

dict_items([('two', 10), ('one', 8), ('three', 7)])

In [25]:
for k,v in type_dict.items():
    if k in merge_df.columns:
        cont = merge_df[k].fillna(0).astype(int).sum()
        if v == cont:
            print(f"{k} - {cont} - correct")
        else:
            print(f"{k} - {cont} - incorrect")
            print(f"For {k} it should be {v}")
    else:
        print(f"{k} - this column doesn't exist")


two - 10 - correct
one - 8 - correct
three - 7 - correct


In [24]:
merge_df.three

0     1.0
1     1.0
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     1.0
9     NaN
10    1.0
11    NaN
12    NaN
13    NaN
14    1.0
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    1.0
21    1.0
22    NaN
23    NaN
24    NaN
Name: three, dtype: float64

In [18]:
merge_df.drop(columns=['three'],inplace=True)

In [11]:
merge_df['balance_amount_x'] = merge_df.apply(lambda row: row['balance_amount_x'] if pd.notnull(row['balance_amount_x']) else row['balance_amount_y'], axis=1)
merge_df['balance_amount'] = merge_df[['balance_amount_x', 'balance_amount_y']].max(axis=1)
merge_df.drop(['balance_amount_x', 'balance_amount_y'], axis=1, inplace=True)

In [12]:
# merge_df[['balance_amount','balance_amount_x', 'balance_amount_y']]

In [13]:
merge_df['age_x'] = merge_df.apply(lambda row: row['age_x'] if pd.notnull(row['age_x']) else row['age_y'], axis=1)
merge_df['age'] = merge_df[['age_x', 'age_y']].max(axis=1)
merge_df.drop(['age_x', 'age_y'], axis=1, inplace=True)

In [14]:
merge_df['dob'] = merge_df['dob_x'].fillna(merge_df['dob_y'])

# Drop the original 'name_x' and 'name_y' columns
merge_df.drop(['dob_x', 'dob_y'], axis=1, inplace=True)


In [15]:
# df[['dob','dob_x','dob_y']]

In [19]:
merge_df['dob'] = pd.to_datetime(merge_df['dob'],dayfirst=True).dt.strftime('%Y-%m-%d')

In [20]:
merge_df.to_excel('test.xlsx',index=False)